<a href="https://colab.research.google.com/github/Pranjal11095/MARS_openproject_2025/blob/main/mars_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-docx PyMuPDF pytesseract pdf2image langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 20.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=1dd2cde81326bd031ccd32df60219d8163103579f53a4f9754ff89e3e0621c6a
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [2]:

import os
import re
import fitz
import pytesseract
from pdf2image import convert_from_path
from docx import Document
from langdetect import detect
from collections import Counter
import json
import mimetypes
import time


In [3]:

def extract_text(path):
    ext = os.path.splitext(path)[-1].lower()
    if ext == ".pdf": return extract_text_from_pdf(path)
    elif ext == ".docx": return extract_text_from_docx(path)
    elif ext == ".txt": return extract_text_from_txt(path)
    else: raise ValueError("Unsupported file format")

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        t = page.get_text()
        if not t.strip(): t = ocr_pdf_page(path, page.number + 1)
        text += t + "\n"
    return text

def ocr_pdf_page(path, page_num):
    images = convert_from_path(path, first_page=page_num, last_page=page_num)
    return pytesseract.image_to_string(images[0])

def extract_text_from_docx(path):
    return "\n".join([p.text for p in Document(path).paragraphs])

def extract_text_from_txt(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()


In [4]:

def preprocess_text(text):
    return re.sub(r"\s+", " ", text).strip()

def extract_title(text):
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    title = clean_lines[0] if clean_lines else "Untitled"
    return title

def extract_keywords(text, num=5):
    words = re.findall(r'\b[a-zA-Z]{5,}\b', text.lower())
    common = Counter(words).most_common(num)
    return [w for w, _ in common]

def summarize_text(text):
    return " ".join(text.split()[:100]) + "..." if len(text.split()) > 100 else text

def detect_language(text):
    try: return detect(text)
    except: return "unknown"

def generate_metadata(text, filename):
    clean = preprocess_text(text)
    stat_info = os.stat(filename)
    return {
        "title": extract_title(clean)[:100],
        "keywords": extract_keywords(clean),
        "summary": summarize_text(clean),
        "language": detect_language(clean),
        "filename": os.path.basename(filename),
        "word_count": len(clean.split()),
        "created_time": time.ctime(stat_info.st_ctime),
        "file_type": mimetypes.guess_type(filename)[0] or "unknown"
    }


In [5]:

from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]

text = extract_text(file_name)
metadata = generate_metadata(text, file_name)

print(json.dumps(metadata, indent=4))


Saving Data Analysis Report.pdf to Data Analysis Report.pdf
{
    "title": "Data Analysis Report 1. Introduction This report presents an in-depth analysis of the Body Shop data",
    "keywords": [
        "analysis",
        "variables",
        "values",
        "numerical",
        "dataset"
    ],
    "summary": "Data Analysis Report 1. Introduction This report presents an in-depth analysis of the Body Shop dataset. The objective was to perform data cleaning, exploratory data analysis (EDA), and identify relationships between variables. The findings aim to enhance understanding of the dataset and reveal patterns that could inform business strategies. 2. Data Cleaning Initial Inspection: \u2022 The dataset structure was examined using df.info() and df.describe(). \u2022 Date columns were converted to datetime format for proper handling. Handling Missing Values: \u2022 Missing values in numerical columns were imputed using the median. \u2022 Categorical columns with missing values wer